In [39]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Homework 1
This is the first homework of the MLOPS zoomcamp, the link to the github project is [here](https://github.com/DataTalksClub/mlops-zoomcamp/blob/main/01-intro/homework.md)
 The aim of this homework is to predict the pickup duration by using pickupLocationID and dropOffLocationId

In [2]:
os.listdir()

['.ipynb_checkpoints',
 'fhv_tripdata_2021-01.parquet',
 'fhv_tripdata_2021-02.parquet',
 'Untitled.ipynb']

#### Loading the data
data are in parquet format, you should install pyarrow to be able to read them with pandas

In [3]:
df1 = pd.read_parquet('fhv_tripdata_2021-01.parquet')
df2 = pd.read_parquet('fhv_tripdata_2021-02.parquet')

#### Working with january file to train the model

In [5]:
print(len(df1))

1154112


In [6]:
df1.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [74]:
df2.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014,92.000000
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021,10.666667
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021,14.566667
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021,7.950000
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037,13.800000


In [7]:
df1.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
dtype: object

In [75]:
df2.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
duration                         float64
dtype: object

In [13]:
# compute the duration
df1['duration'] = (df1.dropOff_datetime - df1.pickup_datetime).dt.total_seconds().div(60).astype(float)

In [16]:
#mean duration
print(df1['duration'].mean())

19.1672240937939


In [19]:
df1.duration.describe()

count    1.154112e+06
mean     1.916722e+01
std      3.986922e+02
min      1.666667e-02
25%      7.766667e+00
50%      1.340000e+01
75%      2.228333e+01
max      4.233710e+05
Name: duration, dtype: float64

In [23]:
df1_new = df1[(df1.duration>=1) & (df1.duration<=60)]
df1_new = df1_new.fillna(-1)

In [27]:
print(100*len(df1_new[df1_new.PUlocationID==-1])/len(df1_new))

83.52732770722618


In [31]:
encoder = OneHotEncoder()
data = df1_new[['PUlocationID', 'DOlocationID']].copy()
X_train = encoder.fit_transform(data)

In [32]:
X_train.shape

(1109826, 525)

In [34]:
lr = LinearRegression()
lr.fit(X_train, df1_new.duration)

LinearRegression()

In [41]:
print(mean_squared_error(df1_new.duration, lr.predict(X_train), squared=False))

10.528519107205662


###  Testing on february file
But there is a problem with the encoder, the shape of the matrix of features for january file is 525 but for february is 526

In [53]:
df2['duration'] = (df2.dropOff_datetime - df2.pickup_datetime).dt.total_seconds().div(60).astype(float)
df2_new = df2[(df2.duration>=1) & (df2.duration<=60)]
df2_new = df2_new.fillna(-1)
X_test = encoder.transform(df2_new[['PUlocationID', 'DOlocationID']])
X_test.shape

(990113, 526)

In [68]:
print(len(set(df1_new['DOlocationID'])))
print(len(set(df2_new['DOlocationID'])))

263
264


In [73]:
set(df2_new['DOlocationID'])-set(df1_new['DOlocationID'])  

{110.0}